<a href="https://colab.research.google.com/github/Kylehanna/Instacart-Market-Basket-Analysis/blob/main/Notebooks/03_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os 
from statsmodels.graphics.mosaicplot import mosaic
color = sns.color_palette()

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

#from library.sb_utils import save_file

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/GROUND GAME AI/00 Capstone 2 - Instacart Market Basket Analysis (Customer Segmentation)/Instacart-Market-Basket-Analysis/data/instacartData_prior.csv')

In [4]:
df.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0
1,1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0
2,2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0
3,3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0
4,4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Unnamed: 0              int64  
 1   order_id                int64  
 2   product_id              int64  
 3   add_to_cart_order       int64  
 4   reordered               int64  
 5   product_name            object 
 6   aisle_id                int64  
 7   department_id           int64  
 8   aisle                   object 
 9   department              object 
 10  user_id                 int64  
 11  eval_set                object 
 12  order_number            int64  
 13  order_dow               int64  
 14  order_hour_of_day       int64  
 15  days_since_prior_order  float64
dtypes: float64(1), int64(11), object(4)
memory usage: 3.9+ GB


In [6]:
# df = df['order_dow'].astype(object)

In [7]:
df.shape

(32434489, 16)

## Feature Engineering

In [8]:
#Total number of orders placed by each users
users = df.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()

In [9]:
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [10]:
# Average number of products bought by the user in each purchase.
#1. First getting the total number of products in each order.
total_prd_per_order = df.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()
total_prd_per_order.head(10)

,user_id,order_id,total_products_per_order
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6
5,1,2398795,6
6,1,2539329,5
7,1,2550362,9
8,1,3108588,6
9,1,3367565,4


In [11]:
#2. Getting the average number of products purchased by each user
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [12]:
#deleting the total_prd_per_order dataframe
del total_prd_per_order

In [13]:
#dow of most orders placed by each user
from scipy import stats
dow = df.groupby(by='user_id')['order_dow'].agg(lambda x: stats.mode(x)[0]).to_frame('dow_most_orders_u').reset_index()
dow.head()

,user_id,dow_most_orders_u
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [14]:
#hour of day when most orders placed by each user
from scipy import stats
hod = df.groupby(by='user_id')['order_hour_of_day'].agg(lambda x: stats.mode(x)[0]).to_frame('hod_most_orders_u').reset_index()
hod.head()

,user_id,hod_most_orders_u
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [15]:
#merging the user created features.

#1. merging avg_products with users
users = users.merge(avg_products, on='user_id', how='left')
#deleting avg_products
del avg_products
users.head()

,user_id,u_num_of_orders,u_avg_prd
0,1,10,5.900000
1,2,14,13.928571
2,3,12,7.333333
3,4,5,3.600000
4,5,4,9.250000


In [16]:
#2. merging dow with users.
users = users.merge(dow, on='user_id', how='left')
#deleting dow
del dow
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,10,5.900000,4
1,2,14,13.928571,2
2,3,12,7.333333,0
3,4,5,3.600000,4
4,5,4,9.250000,3


In [17]:
#number of times a product was purchased.
prd = df.groupby(by='product_id')['order_id'].agg('count').to_frame('prd_count_p').reset_index()
prd.head()

,product_id,prd_count_p
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [18]:
#products reorder ratio.
reorder_p = df.groupby(by='product_id')['reordered'].agg('mean').to_frame('p_reordered_ratio').reset_index()
reorder_p.head()

,product_id,p_reordered_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [19]:
#merging the reorder_p with prd
prd = prd.merge(reorder_p, on='product_id', how='left')
#deleting reorder_p
del reorder_p
prd.head()

,product_id,prd_count_p,p_reordered_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.600000


In [20]:
#how many times has a user bought the same product?
uxp = df.groupby(by=['user_id', 'product_id'])['order_id'].agg('count').to_frame('uxp_times_bought').reset_index()
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [21]:
#reorder ratio of the user for each product.
reorder_uxp = df.groupby(by=['user_id', 'product_id'])['reordered'].agg('mean').to_frame('uxp_reordered_ratio').reset_index()
reorder_uxp.head()

,user_id,product_id,uxp_reordered_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [22]:
#merging the two dataframes into one
uxp = uxp.merge(reorder_uxp, on=['user_id', 'product_id'], how='left')
#deleting reorder_uxp
del reorder_uxp
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio
0,1,196,10,0.900000
1,1,10258,9,0.888889
2,1,10326,1,0.000000
3,1,12427,10,0.900000
4,1,13032,3,0.666667


In [23]:
#merging users df into uxp
data = uxp.merge(users, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,196,10,0.900000,10,5.9,4
1,1,10258,9,0.888889,10,5.9,4
2,1,10326,1,0.000000,10,5.9,4
3,1,12427,10,0.900000,10,5.9,4
4,1,13032,3,0.666667,10,5.9,4


In [24]:
#merging products df into data
data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
0,1,196,10,0.900000,10,5.9,4,35791,0.776480
1,1,10258,9,0.888889,10,5.9,4,1946,0.713772
2,1,10326,1,0.000000,10,5.9,4,5526,0.652009
3,1,12427,10,0.900000,10,5.9,4,6476,0.740735
4,1,13032,3,0.666667,10,5.9,4,3751,0.657158


In [25]:
del [users, prd, uxp]

In [26]:
df_train = pd.read_csv('/content/drive/MyDrive/GROUND GAME AI/00 Capstone 2 - Instacart Market Basket Analysis (Customer Segmentation)/Instacart-Market-Basket-Analysis/data/instacartData_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/GROUND GAME AI/00 Capstone 2 - Instacart Market Basket Analysis (Customer Segmentation)/Instacart-Market-Basket-Analysis/data/instacartData_test.csv')

In [27]:
df_train.shape

(1384617, 16)

In [28]:
df_train.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,112108,train,4,4,10,9.0
2,2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
3,3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
4,4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0


In [29]:
df_test.shape

(75000, 8)

In [30]:
#keeping only the train and test from eval_set orders dataframe.

order_future = df_train.loc[((df_train.eval_set == 'train') | (df_test.eval_set == 'test')), ['user_id', 'eval_set', 'order_id']]
order_future.head()


,user_id,eval_set,order_id
0,112108,train,1
1,112108,train,1
2,112108,train,1
3,112108,train,1
4,112108,train,1


In [31]:
order_future['eval_set'].nunique()

1

In [ ]:
data = data.merge(order_future, on='user_id', how='left')
data.head()

# Save preprocessed data in new csv

In [ ]:
'''# save the data to a new csv file
#datapath = '../../data'
#save_file(df, 'processed.csv', datapath)

df.to_csv('processed_data.csv',index=False)'''